In [76]:
%load_ext autoreload
%autoreload 2 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import findspark
findspark.init()

In [3]:
from random import random
from operator import add
from pyspark.sql import SparkSession
import pandas as pd


DATA_DIR = '/home/bob/gtd/Iconic/data/samples/'

In [120]:
affs_country = pd.read_csv('/home/bob/gtd/Iconic/data/Affiliations_FC.txt', sep = '\t')
affs_country = affs_country.drop(columns = ['Unnamed: 0'])
# affs_country['country'] = affs_country['country'].astype(str).apply(lambda string : string[1:-1].replace("'",""))
# affs_country['countryCode'] = affs_country['country'].apply(lambda string : "".join(string.split(',')[:-1]))
# affs_country['country'] = affs_country['country'].apply(lambda string : "".join(string.split(',')[-1:]))

In [121]:
affs_country

,id,rank,name,dname,grid,page,wiki,papers,citations,created_at,country,countryCode
0,20455151,9759,air liquide,Air Liquide,grid.476009.c,https://web.archive.org/web/20100205175402/htt...,http://en.wikipedia.org/wiki/Air_Liquide,6314,44828,2016-06-24,Grande Prairie,CAN
1,24386293,13640,hellenic national meteorological service,Hellenic National Meteorological Service,NaN,http://www.hnms.gr/hnms/english/index_html,http://en.wikipedia.org/wiki/Hellenic_National...,81,1531,2016-06-24,South East,BWA
2,32956416,13117,catholic university of the west,Catholic University of the West,grid.448708.7,http://www.uco.fr/,http://en.wikipedia.org/wiki/Catholic_Universi...,236,2565,2016-06-24,Fort Worth,USA
3,35926432,12053,mackay medical college,Mackay Medical College,grid.452449.a,http://www.mmc.edu.tw/,http://en.wikipedia.org/wiki/Mackay_Medical_Co...,815,6080,2016-06-24,三芝區,TWN
4,37448385,11782,chinese people s public security university,Chinese People's Public Security University,NaN,http://www.ppsuc.edu.cn/,http://en.wikipedia.org/wiki/People's_Public_S...,1502,1195,2016-06-24,University,USA
5,37712293,13170,madeira interactive technologies institute,Madeira Interactive Technologies Institute,NaN,http://www.m-iti.org/,http://en.wikipedia.org/wiki/Madeira_Interacti...,296,1409,2016-06-24,Johannesburg,ZAF
6,42117203,16960,queen s college london,"Queen's College, London",NaN,http://www.qcl.org.uk/,"http://en.wikipedia.org/wiki/Queen's_College,_...",8,10,2016-06-24,London,GBR
7,50614327,11813,amec foster wheeler,Amec Foster Wheeler,grid.480159.1,http://www.amecfw.com/,http://en.wikipedia.org/wiki/Amec_Foster_Wheeler,951,7423,2016-06-24,Chennai,IND
8,62987803,16029,ukrainian state university of finance and inte...,Ukrainian State University of Finance and Inte...,grid.445773.6,http://usufit.edu.ua/,http://en.wikipedia.org/wiki/Ukrainian_State_U...,21,18,2016-06-24,Xalapa,MEX
9,71576600,14540,ural state mining university,Ural State Mining University,grid.446243.3,http://www.ursmu.ru/,http://en.wikipedia.org/wiki/Ural_State_Mining...,89,78,2016-06-24,University,USA


In [123]:
affs_country.to_csv('/home/bob/gtd/Iconic/data/Affiliations_FC.txt', sep = '\t', index = False)

In [4]:
spark = SparkSession\
    .builder\
    .appName("IconicSparkPy")\
    .getOrCreate()

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [73]:
def read_dataframe(path):
    return spark.read.csv(DATA_DIR + path, sep = '\t', schema = affs_schema, mode = 'PERMISSIVE')


In [74]:
affs_schema = "id STRING, rank STRING, name STRING, dname STRING, grid STRING,\
    page STRING, wiki  STRING, papers  STRING, citations  STRING,\
	lat STRING, long STRING, createdAt STRING"

In [75]:
affs = read_dataframe('Affiliations.txt')
affs

DataFrame[id: string, rank: string, name: string, dname: string, grid: string, page: string, wiki: string, papers: string, citations: string, lat: string, long: string, createdAt: string]

In [70]:
s = affs.take(10)[3][0]
s
s.split('\t')
len(s.split('\t'))

'107371206\t12638\tsao paulo federal institute of education science and technology\tSão Paulo Federal Institute of Education'

['107371206',
 '12638',
 'sao paulo federal institute of education science and technology',
 'São Paulo Federal Institute of Education']

4

In [ ]:
papers = read_dataframe('Papers.txt')
authors = read_dataframe('Authors.txt')
fos = read_dataframe('FieldsOfStudy.txt')
affs = read_dataframe('Affiliations.txt')
paa = read_dataframe('PaperAuthorAffiliations.txt')
pf = read_dataframe('PaperFieldsOfStudy.txt')
pr = read_dataframe('PaperReferences.txt')
fc = read_dataframe('FieldOfStudyChildren.txt')

# Preliminaries
## Get root field of study 

In [6]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [7]:
def computeRootFos( fieldId):
    filtered = fc.filter(col("child") == fieldId)
#     print(filtered.take(1))
    if len(filtered.take(1)) == 0:
        return fieldId
    parent = filtered.take(1)[0][0]
    if parent == fieldId:
#         print(parent)
        return parent
    else:
#         print('rec')
        return computeRootFos( parent)

In [8]:
computeRootFos(8105449)

12843

add the root field of study to each paper

In [9]:
from pyspark.sql import Row
pf.rdd.map(lambda row : Row(paper = row[0], fos = row[1], similarity = row[2])).toDF()
fields = pf.select("fos").distinct().collect()
fields = map(lambda field : {'fos' : field[0], 'root' :computeRootFos(field[0])}, fields)
fields = list(fields)

DataFrame[fos: bigint, paper: bigint, similarity: double]

In [10]:
fr = pd.DataFrame(fields)

define helper function to use the root dataframe for getting the root of a field id

In [12]:
from pyspark.sql.functions import udf
def getRootFos(fos):
    return fr[fr["fos"] == fos]["root"][0]

rootFos_udf = udf(lambda fos : getRootFos(fos))

In [13]:
from pyspark.sql.functions import lit
rootFos_udf(lit(124952713))
getRootFos(124952713)

Column<b'<lambda>(124952713)'>

124952713

## Aggregated new data

In [14]:
# ego country
authorDetails = paa.join(pf, "paper")\
    .select("author", "paper","affiliation", "fos")\
    .rdd.map(lambda row : Row(author = row[0], paper = row[1], affiliation = row[2], fos = getRootFos(row[3]))).toDF()\
    .groupBy("author", "fos", "affiliation")\
    .agg(F.countDistinct("paper"))\
    .join(MAG.dfAff, col("affiliation") == col("id"))\
    .select("author","fos","countryCode","country")

authorDetails

ValueError: RDD is empty

# Filtered tables

In [15]:
# FILTERED_PAA_YEARS
paa = paa.join(papers, col("id") == col("paper"))\
    .filter( (col("year") >= 2007) & (col("year") <= 2017))\
    .select("paper", "author", "affiliation")
paa

DataFrame[paper: int, author: bigint, affiliation: bigint]

In [16]:
# PF_WITH_ROOT
pf = pf.withColumn("root", rootFos_udf("fos"))
pf

DataFrame[paper: int, fos: bigint, similarity: double, root: string]

In [17]:
fos = 10
# FILTERED_PAA_FOS
paa = paa.join(pf, "paper")\
    .filter(col("root") == fos)\
    .select("paper", "author", "affiliation")
paa

DataFrame[paper: int, author: bigint, affiliation: bigint]

## Add test data

In [18]:
from pyspark.sql import Row

In [55]:
taffs = [(1, "RO") , (2, "EN"), (3, "IT")]
rdd = spark.sparkContext.parallelize(taffs)
rdd = rdd.map(lambda x : Row(id = x[0], countryCode = x[1]))
taffs = spark.createDataFrame(rdd)
taffs

DataFrame[countryCode: string, id: bigint]

In [56]:
tpaa = [(1, 2, 1),(1,3, 1),(2,3, 1),(2,1, 1),(3,2, 1),(1,5, 2),(3,3, 1),(3,5, 2),(4,7, 3)]
rdd = spark.sparkContext.parallelize(tpaa)
rdd = rdd.map(lambda x: Row(paper=int(x[0]), author=int(x[1]), affiliation = x[2]))
tpaa = spark.createDataFrame(rdd)
tpaa

DataFrame[affiliation: bigint, author: bigint, paper: bigint]

In [57]:
tpr = [(1,2), (3, 2), (3,4), (4, 1)]
rdd = spark.sparkContext.parallelize(tpr)
rdd = rdd.map(lambda x: Row(citing=int(x[0]), cited=int(x[1])))
tpr = spark.createDataFrame(rdd)
tpr

DataFrame[cited: bigint, citing: bigint]

In [58]:
tpapers = [ (1, 2007), (2,2008), (3,2008), (4,2009) ]
rdd = spark.sparkContext.parallelize(tpapers)
rdd = rdd.map(lambda x : Row(id = x[0], year = x[1]))
tpapers = spark.createDataFrame(rdd)
tpapers

DataFrame[id: bigint, year: bigint]

In [90]:
print('PAPER AUTHOR AFF')
tpaa.sort("author").collect()
print('PAPER REFERENCES')
tpr.sort("cited").collect()
print('PAPERS')
tpapers.collect()
print('COUNTRIES')
ego_country.select("author", "countryCode").distinct().collect()
print('COAUTHORS')
paa.alias("1").join(paa.alias("2"), col("1.paper") == col("2.paper"))\
    .filter(col("1.author") != col("2.author"))\
    .select(col("1.author").alias("ego"), col("2.author").alias("alter"))\
    .sort("ego")\
    .distinct()\
    .collect()

PAPER AUTHOR AFF


[Row(affiliation=1, author=1, paper=2),
 Row(affiliation=1, author=2, paper=1),
 Row(affiliation=1, author=2, paper=3),
 Row(affiliation=1, author=3, paper=1),
 Row(affiliation=1, author=3, paper=2),
 Row(affiliation=1, author=3, paper=3),
 Row(affiliation=2, author=5, paper=1),
 Row(affiliation=2, author=5, paper=3),
 Row(affiliation=3, author=7, paper=4)]

PAPER REFERENCES


[Row(cited=1, citing=4),
 Row(cited=2, citing=1),
 Row(cited=2, citing=3),
 Row(cited=4, citing=3)]

PAPERS


[Row(id=1, year=2007),
 Row(id=2, year=2008),
 Row(id=3, year=2008),
 Row(id=4, year=2009)]

COUNTRIES


[Row(author=3, countryCode='RO'),
 Row(author=1, countryCode='RO'),
 Row(author=7, countryCode='IT'),
 Row(author=5, countryCode='EN'),
 Row(author=2, countryCode='RO')]

COAUTHORS


[Row(ego=1, alter=3),
 Row(ego=2, alter=3),
 Row(ego=2, alter=5),
 Row(ego=3, alter=2),
 Row(ego=3, alter=5),
 Row(ego=3, alter=1),
 Row(ego=5, alter=2),
 Row(ego=5, alter=3)]

## New table head

In [60]:
paa = tpaa
pr = tpr
papers = tpapers
affs = taffs

In [67]:
# PAA_country
paa_country = paa.join(affs, col("affiliation") == col("id"))\
    .select("author", "paper", "affiliation", "countryCode")
paa_country.collect()

[Row(author=2, paper=1, affiliation=1, countryCode='RO'),
 Row(author=3, paper=1, affiliation=1, countryCode='RO'),
 Row(author=3, paper=2, affiliation=1, countryCode='RO'),
 Row(author=1, paper=2, affiliation=1, countryCode='RO'),
 Row(author=2, paper=3, affiliation=1, countryCode='RO'),
 Row(author=3, paper=3, affiliation=1, countryCode='RO'),
 Row(author=7, paper=4, affiliation=3, countryCode='IT'),
 Row(author=5, paper=1, affiliation=2, countryCode='EN'),
 Row(author=5, paper=3, affiliation=2, countryCode='EN')]

In [155]:
# EGO_Country
ego_country = paa_country.select("author", "countryCode").distinct()
ego_country.collect()

[Row(author=3, countryCode='RO'),
 Row(author=1, countryCode='RO'),
 Row(author=7, countryCode='IT'),
 Row(author=5, countryCode='EN'),
 Row(author=2, countryCode='RO')]

In [118]:
# EGO_papers
ego_papers = paa.groupBy(col("author").alias("ego")).count().withColumnRenamed("count","papers")
ego_papers.collect()

[Row(ego=7, papers=1),
 Row(ego=5, papers=2),
 Row(ego=1, papers=1),
 Row(ego=3, papers=3),
 Row(ego=2, papers=2)]

In [94]:
#EGO_citations
ego_citations = paa.join(pr, col("paper") == col("cited"))\
    .groupBy(col("author").alias("ego"))\
    .count().withColumnRenamed("count","citations")\

ego_citations.collect()    

[Row(ego=7, citations=1),
 Row(ego=5, citations=1),
 Row(ego=1, citations=2),
 Row(ego=3, citations=3),
 Row(ego=2, citations=1)]

In [139]:
# EGO_PAIRS - paa join paa
paa2 = paa.alias("1").join(paa.alias("2"), col("1.paper") == col("2.paper"))\
    .filter(col("1.author") != col("2.author"))

In [143]:
#EGO_coauthors
from pyspark.sql.functions import countDistinct
paa2.groupBy("1.author")\
    .agg(F.countDistinct("2.author").alias("coauthors"))\
    .collect()

[Row(author=5, coauthors=2),
 Row(author=1, coauthors=1),
 Row(author=3, coauthors=3),
 Row(author=2, coauthors=2)]

In [145]:
# COAUTHORS
coauth = paa2\
    .select(col("1.author").alias("ego"), col("2.author").alias("alter"))\
    .distinct()
coauth.collect()

[Row(ego=3, alter=5),
 Row(ego=1, alter=3),
 Row(ego=2, alter=5),
 Row(ego=2, alter=3),
 Row(ego=5, alter=2),
 Row(ego=3, alter=2),
 Row(ego=5, alter=3),
 Row(ego=3, alter=1)]

In [146]:
#EGO_edges - numarul de ani in care am colaborat cu coautorii ? 
from pyspark.sql.functions import countDistinct
paa2.join(papers, col("id") == col("1.paper"))\
    .groupBy("1.author")\
    .agg(F.countDistinct("year").alias("years_collaborated"))\
    .collect()

[Row(author=5, years_collaborated=2),
 Row(author=1, years_collaborated=1),
 Row(author=3, years_collaborated=2),
 Row(author=2, years_collaborated=2)]

In [153]:
# ALTER_domestic
paa = paa_country
paa2.filter(col("1.countryCode") == col("2.countryCode"))\
    .groupBy("1.author")\
    .agg(F.countDistinct("2.author").alias("domestics"))\
    .collect()

[Row(author=1, domestics=1),
 Row(author=3, domestics=2),
 Row(author=2, domestics=1)]

In [149]:
# ALTER_nondomestic
# paa = ego_country
paa = tpaa
paa2.filter(col("1.countryCode") != col("2.countryCode"))\
    .groupBy("1.author")\
    .agg(F.countDistinct("2.author").alias("nondomestics"))\
    .collect()

[Row(author=5, nondomestics=2),
 Row(author=3, nondomestics=1),
 Row(author=2, nondomestics=1)]

In [137]:
# ALTER_citations - including ego's publications 
# we will substract ALTER_EGO_CITATIONS with_coauthors afterwards 
# # if we want to get ALTER_CITATIONS not including ego's publications
from pyspark.sql.functions import count
coauth.alias("1").join(ego_citations.alias("2"), col("1.alter") == col("2.ego"))\
    .groupBy(col("1.ego"))\
    .agg(F.sum("citations").alias("alter_citations"))\
    .collect()
    

[Row(ego=5, alter_citations=4),
 Row(ego=1, alter_citations=3),
 Row(ego=3, alter_citations=4),
 Row(ego=2, alter_citations=4)]

In [127]:
from pyspark.sql.functions import count, sum
#ALTER_papers - count total number of papers written by coauthors, including ego's publications
# # we can substract like above (ALTER_EGO_PAPERS)
coauth.alias("1").join(ego_papers.alias("2"), col("1.alter") == col("2.ego"))\
    .groupBy("1.ego")\
    .agg(F.sum("papers").alias("alter_papers"))\
    .collect()

[Row(ego=5, alter_papers=5),
 Row(ego=1, alter_papers=3),
 Row(ego=3, alter_papers=5),
 Row(ego=2, alter_papers=5)]

For below, we need a join that takes left elements that do not meet the condition with 0 default value

In [150]:
# ALTER_EGO_papers
paa2.groupBy("1.author")\
    .agg(F.countDistinct("1.paper").alias("papers_not_alone"))\
    .collect()

[Row(author=5, papers_not_alone=2),
 Row(author=1, papers_not_alone=1),
 Row(author=3, papers_not_alone=3),
 Row(author=2, papers_not_alone=2)]

In [151]:
# ALTER_EGO_citations
paa2.join(pr, col("1.paper") == col("cited"))\
    .groupBy("1.author")\
    .agg(F.countDistinct("citing").alias("citations"))\
    .collect()

[Row(author=5, citations=1),
 Row(author=1, citations=2),
 Row(author=3, citations=3),
 Row(author=2, citations=1)]

In [152]:
# ALTER_country
paa2.groupBy("1.author")\
    .agg(F.countDistinct("2.countryCode").alias("countries"))\
    .collect()

[Row(author=5, countries=1),
 Row(author=1, countries=1),
 Row(author=3, countries=2),
 Row(author=2, countries=2)]

In [136]:
# ALTER_MAX_papers
coauth.alias("1").join(ego_papers.alias("2"), col("1.alter") == col("2.ego"))\
    .groupBy("1.ego")\
    .agg(F.max("papers").alias("max_alter_papers"))\
    .collect()

[Row(ego=5, max_alter_papers=3),
 Row(ego=1, max_alter_papers=3),
 Row(ego=3, max_alter_papers=2),
 Row(ego=2, max_alter_papers=3)]

In [135]:
# ALTER_MAX_citations
coauth.alias("1").join(ego_citations.alias("2"), col("1.alter") == col("2.ego"))\
    .groupBy(col("1.ego"))\
    .agg(F.max("citations").alias("max_alter_citations"))\
    .collect()

[Row(ego=5, max_alter_citations=3),
 Row(ego=1, max_alter_citations=3),
 Row(ego=3, max_alter_citations=2),
 Row(ego=2, max_alter_citations=3)]